In [1]:
from aiida import load_profile
load_profile()

Profile<uuid='468a9d4d0812492ba52462f2aca53e23' name='presto'>

In [2]:
# from aiida.orm import StructureData
# from ase.build import bulk
# from ase.io import read

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
# structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [3]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [4]:
from aiida.orm import load_code
code = load_code("janus@localhost")

In [5]:
from aiida.orm import Str, Float, Bool
inputs = {
    "code": code,
    "model": model,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "fmax": Float(0.1), 
    "opt_cell_lengths": Bool(False), 
    "opt_cell_fully": Bool(True), 
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

In [6]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")

In [7]:
from aiida_workgraph import WorkGraph
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

wg = WorkGraph("Calculation Workgraph")
wg.inputs = inputs

num_structs = len(read("lj-traj.xyz", index=":"))

for i in range(num_structs - 9):

    structure = StructureData(ase=read("lj-traj.xyz", index=f"{i}"))
    
    geomopt_calc = wg.add_task(
        geomoptCalc,
        code = wg.inputs.code,
        model = wg.inputs.model,
        arch = wg.inputs.arch,
        precision = wg.inputs.precision,
        device = wg.inputs.device,
        metadata = wg.inputs.metadata,
        fmax = wg.inputs.fmax,
        opt_cell_lengths = wg.inputs.opt_cell_lengths,
        opt_cell_fully = wg.inputs.opt_cell_fully,
        struct = structure,
    )

    descriptors_calc = wg.add_task(
        descriptorsCalc,
        code = wg.inputs.code,
        model = wg.inputs.model,
        arch = wg.inputs.arch,
        precision = wg.inputs.precision,
        device = wg.inputs.device,
        metadata = wg.inputs.metadata,
        struct = geomopt_calc.outputs.final_structure,
        calc_per_element = True
    )

    wg.outputs.final_structures = (
        {f"structs{i}" : descriptors_calc.outputs.xyz_output}
        )
    


defining outputnode
defining outputnode


In [8]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Calcula…

In [9]:
wg.run()

07/28/2025 02:33:32 PM <86233> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
07/28/2025 02:33:32 PM <86233> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
07/28/2025 02:33:32 PM <86233> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
07/28/2025 02:33:36 PM <86233> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9494|WorkGraphEngine|continue_workgraph]: tasks ready to run: calcjob,calcjob2
07/28/2025 02:33:39 PM <86233> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9494|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 9498, 9502
07/28/2025 02:34:02 PM <86233> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9494|WorkGraphEngine|update_task_state]: Task: calcjob, type: CALCJOB, finished.
07/28/2025 02:34:03 PM

In [11]:
import shutil, os
from pathlib import Path

descriptors_output = list(wg.outputs.final_structures)
traj_structs = []

for i in range(len(descriptors_output)):

    # create list variable which holds output structs
    singlefiledata = wg.outputs.final_structures[f"structs{i}"]
    traj_structs.append(singlefiledata.value)

    # create file which holds output structs
    with singlefiledata.value.open(mode='rb') as source:
        with open(f"structs.xyz", mode='ab') as target:
            shutil.copyfileobj(source, target)

print(traj_structs)

[<SinglefileData: uuid: 6067babe-6e0f-48e5-a6ab-d05f8239cfcb (pk: 9535)>, <SinglefileData: uuid: 40b168c7-c2a0-44f5-994b-795b88c1ae75 (pk: 9539)>]


## Testing output

In [ ]:
# # test janus output

# ! janus descriptors --struct lj-traj.xyz --arch mace_mp --calc-per-element

In [ ]:
# from aiida.plugins import DataFactory
# from aiida import load_profile, profile_context, orm

# load_profile()

# file = orm.load_node(9364).outputs.xyz_output
# file1 = orm.load_node(9367).outputs.xyz_output

# traj_structs = [file, file1]
# print(traj_structs)

In [ ]:
from ase.io import read, write

traj_file = '/home/mtr46585/aiida-mlip/examples/tutorials/janus_results/lj-traj-descriptors.extxyz'

x = read(traj_file, index=":")[0]

print(x.info)


In [13]:
from sample_split import process_and_split_data


process_and_split_data(
    trajectory_path = traj_structs,
    config_types = "",
    n_samples = 2,
    prefix = "",
    scale = 1.0e5,
    append_mode = False
    )


create files: train_file=PosixPath('train.xyz'), valid_file=PosixPath('valid.xyz') and test_file=PosixPath('test.xyz')
Processing: ('all', 'aiida'), 2 frames
  ('all', 'aiida'): total=2, train_target=1, vt_target=1
Found 0 structures that were too similar during sampling.


/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 2
  warnings.warn(f"k is too large, set to {n_pts}")
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 1
  warnings.warn(f"k is too large, set to {n_pts}")
